In [1]:
import torch 
from torch import nn
import torch.nn.functional as F

In [2]:
class MyConv2D(nn.Module):
  def __init__(self,in_channels=1,out_channels=1, kernel_size=(1,1), stride=1, padding=0):    
    super(MyConv2D).__init__()
    self.kernel_size = kernel_size
    self.stride = stride
    self.pad = padding
    self.out_channels = out_channels
    self.in_channels = in_channels
    self.weight = torch.rand((out_channels, *kernel_size))
    self.bias = torch.rand(kernel_size)
    print(self.weight)

  def forward(self, x):
    x_shape = x.shape
    x_pad = x
    if self.pad:
      x_pad = torch.zeros(x_shape[0], x_shape[1], x_shape[2]+2*self.pad, x_shape[3]+2*self.pad)
      x_pad[:,:,self.pad:-self.pad, self.pad:-self.pad] = x

    k = self.kernel_size
    
    n_h = 1 + int( (x_pad.shape[2]-k[0])/self.stride )
    n_w = 1 + int( (x_pad.shape[3]-k[1])/self.stride )

    y = torch.zeros((x_shape[0], self.out_channels, n_h, n_w))

    for i in range((x_pad.shape[2]- k[0])//self.stride+1):
      idx= i * self.stride
      for j in range((x_pad.shape[3]- k[1])//self.stride+1):
        
        jdx =j * self.stride
        tmp = x_pad[:,:,idx:idx+k[0],jdx:jdx+k[1]]
        r = tmp * self.weight + self.bias
        value = torch.sum(r, dim=(2,3))
        y[:,:,i,j] = value  
           
    return y

In [3]:
x = torch.rand((1,1,5,5))
x

tensor([[[[0.2488, 0.2807, 0.3279, 0.5437, 0.4571],
          [0.1822, 0.1355, 0.7006, 0.6699, 0.6017],
          [0.6634, 0.2642, 0.4713, 0.8826, 0.5040],
          [0.8504, 0.2173, 0.4173, 0.8770, 0.2549],
          [0.0097, 0.7825, 0.5828, 0.0092, 0.4350]]]])

In [4]:
d = MyConv2D(1, 1, kernel_size=(3,3),padding=1, stride=1)

tensor([[[0.8039, 0.5660, 0.3056],
         [0.8969, 0.7697, 0.3511],
         [0.2858, 0.7888, 0.8912]]])


In [5]:
y = d.forward(x)
y

tensor([[[[4.6646, 5.4477, 5.9935, 6.2480, 5.6156],
          [5.2833, 5.9009, 6.8173, 7.4564, 6.5196],
          [5.7224, 6.2975, 6.9027, 7.5535, 6.6203],
          [6.0021, 7.1529, 6.3745, 6.8428, 6.4334],
          [4.9400, 5.8598, 5.9426, 5.7024, 5.3024]]]])

In [6]:
conv = nn.Conv2d(1,1,kernel_size=3,stride=1,padding=1)

In [7]:
conv(x)

tensor([[[[-0.1510,  0.0227, -0.1277, -0.0240, -0.0976],
          [-0.1540,  0.0009, -0.2125, -0.3095, -0.0476],
          [-0.3280, -0.0876, -0.2094, -0.4355, -0.0075],
          [-0.0980, -0.2953, -0.3581, -0.1302, -0.2260],
          [-0.1824, -0.4757, -0.4496, -0.1857, -0.1545]]]],
       grad_fn=<ConvolutionBackward0>)